**Necessary Imports**

In [0]:
import numpy as np
import pandas as pd
from scipy.fftpack import dct
import matplotlib.pyplot as plt
import os
import shutil
%matplotlib inline

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


**Channel and Emotion Mappings**

In [0]:
channels = {0:'Fp1', 1:'AF3', 2:'F3', 3:'F7', 4:'FC5', 5:'FC1', 6:'C3', 7:'T7', 8:'CP5', 9:'CP1', 10:'P3',
       11:'P7', 12:'PO3', 13:'O1', 14:'Oz', 15:'Pz', 16:'Fp2', 17:'AF4', 18:'Fz', 19:'F4', 20:'F8', 21:'FC6',
       22:'FC2', 23:'Cz', 24:'C4', 25:'T8', 26:'CP6', 27:'CP2', 28:'P4', 29:'P8', 30:'PO4', 31:'O2'}
emotions = {0:"Neutral", 1:"Anger", 2:"Disgust", 3:"Fear", 4:"Happy", 5:"Sad", 6:"Surprise", 11:"Amusement", 12:"Anxiety"}

path = os.path.join('/content/drive/Shared drives/DSCI400_Revamp','DCT Plots')
try:
  os.mkdir(path)
except:
  print('Overwriting the folder')
  shutil.rmtree(path)
  os.mkdir(path)

**Helper Function to Sort Each DCT by Channel and Emotion**

In [0]:
def plot_dcts(lower_freq, upper_freq):
  """
  This function loads the EEG data and plots the DCT of each EEG from the range
  [lower, upper] inclusive.
  """
  # First, initialize the nested dictionary to contain all possible pairs of
  # channels and emotions. data will store each set of DCT's as a list of numpy 
  # arrays
  data = {}
  c = channels.values()
  e = emotions.keys()
  for key1 in c:
    data[key1] = {}
    for key2 in e:
      data[key1][key2] = []
  
  # Read each session and determine which emotion an individual felt in session
  # i; then, take the DCT for each channel and sort accordingly
  for i in range(2,4000,2): #Hard coded - will need to fix at some point
    try:
      comp = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Label_Data.h5', key = 't' + str(i) + '_labels').transpose()
      comp.index = ['t' + str(i) + '_labels']
      df = pd.read_hdf('/content/drive/Shared drives/DSCI400_Revamp/Waveform_Vocoded_Equalized_Data.h5', 't' + str(i)).transpose()
    except:
      continue
    emotion_val = int(comp.loc['t' + str(i) + '_labels', 'feltEmo'])
    for chan in c:
      vector = df.loc[chan,:].to_numpy()
      transform = dct(vector)
      transform = transform[lower_freq:upper_freq+1]
      data[chan][emotion_val].append(transform)
  return data

**Main Function to Plot and Save DCT's Grouped By Channel and Emotion**



Includes performance measurements with the "time" library

In [0]:
import time

def run_and_plot(lower_freq = 2, upper_freq = 30):
  """
  This function plots the all DCT's grouped by channel and emotion. The function
  defaults to plotting and saving the DCT's between 2 Hz and 30 Hz, inclusive.
  """
  # Phase 1: Get the data of all the DCT's grouped by channel and emotion
  print('Beginning Phase 1')
  start_phase1 = time.time()
  channel_emotion_pairs = plot_dcts(lower_freq, upper_freq)
  end_phase1 = time.time()
  print('Phase 1 completed in ', end_phase1 - start_phase1, ' seconds')

  # Phase 2: Plot each group of DCT's on the same plot and save it to the Google
  # Drive
  print('Beginning Phase 2')
  start_phase2 = time.time()
  for key1 in channel_emotion_pairs:
    for key2 in channel_emotion_pairs[key1]:
      matrix = channel_emotion_pairs[key1][key2]
      fig = plt.figure()
      for vec in matrix:
        plt.scatter(range(lower_freq, upper_freq+1), vec)
      plt.plot(range(lower_freq - 1, upper_freq + 2), [0]*(upper_freq + 3 - lower_freq), 'k-', alpha = 0.6)
      plt.xlabel('Cosine Frequency (Hz)')
      plt.ylabel('Magnitude')
      plt.title('Channel ' + key1 + ' for the Emotion \"' + emotions[key2] + '\"')
      fig.savefig('/content/drive/Shared drives/DSCI400_Revamp/DCT Plots/Channel ' + key1 + ';' + emotions[key2] + ' Emotion')
      plt.close()
  end_phase2 = time.time()
  print('Phase 2 completed in ', end_phase2 - start_phase2, ' seconds')
  print('Overall process completed in ', end_phase2 - start_phase1, ' seconds')

In [0]:
run_and_plot()

Beginning Phase 1
Phase 1 completed in  48.76813364028931  seconds
Beginning Phase 2
Phase 2 completed in  87.50965189933777  seconds
Overall process completed in  136.27866172790527  seconds
